<a href="https://colab.research.google.com/github/muzahid120/Deeplearning/blob/Python-project/CNN/transfer_learning/data%20augmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import tensorflow
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense,Flatten
from keras.applications.vgg16 import VGG16
from keras.preprocessing.image import ImageDataGenerator

In [9]:
import os
import shutil

# Define paths
kaggle_dir = os.path.expanduser('~/.kaggle')
kaggle_json_src = 'kaggle.json'
kaggle_json_dest = os.path.join(kaggle_dir, 'kaggle.json')

# Create the .kaggle directory if it doesn't exist
if not os.path.exists(kaggle_dir):
    os.makedirs(kaggle_dir)

# Move the kaggle.json file to the .kaggle directory
shutil.move(kaggle_json_src, kaggle_json_dest)

# Set permissions for kaggle.json file
os.chmod(kaggle_json_dest, 0o600)

In [10]:
# Specify the directory path
directory_path = os.path.expanduser('~/.kaggle')

# Create the directory if it doesn't exist
os.makedirs(directory_path, exist_ok=True)


In [11]:
! kaggle datasets download -d salader/dogs-vs-cats

 99% 1.06G/1.06G [00:13<00:00, 117MB/s]
100% 1.06G/1.06G [00:13<00:00, 83.4MB/s]


In [12]:
import zipfile
def unzip_file(zip_file_path, extract_to_path):
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        zip_ref.extractall(extract_to_path)
        zip_ref.close()


In [13]:
zip_file_path = '/content/dogs-vs-cats.zip'
extract_to_path = '/content'

unzip_file(zip_file_path, extract_to_path)

In [15]:
batch_size=16
# test_data_genoratore
train_gen=ImageDataGenerator(
    rescale=1./256,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True

)

#test_data_generatore

test_gen=ImageDataGenerator(
    rescale=1./256
)


# train data generatore from directory
train_gen_data=train_gen.flow_from_directory('/content/train',
                              target_size=(224,224),
                              batch_size=batch_size,
                              class_mode='binary')
# test data genoratore from directory

test_gen_data=test_gen.flow_from_directory('/content/test',
                                           target_size=(224,224),
                                           batch_size=batch_size,
                                           class_mode='binary')


Found 20000 images belonging to 2 classes.
Found 5000 images belonging to 2 classes.


In [16]:
conv_base=VGG16(include_top=False,
                weights='imagenet',
                input_shape=(224,224,3))

58889256/58889256 [==============================] - 0s 0us/step


In [17]:
conv_base.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [19]:
conv_base.trainable=False
conv_base.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [22]:
model=Sequential()
model.add(conv_base)
model.add(Flatten())

model.add(Dense(256,activation='relu'))
model.add(Dense(1,activation='sigmoid'))

In [23]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 7, 7, 512)         14714688  
                                                                 
 flatten (Flatten)           (None, 25088)             0         
                                                                 
 dense (Dense)               (None, 256)               6422784   
                                                                 
 dense_1 (Dense)             (None, 1)                 257       
                                                                 
Total params: 21137729 (80.63 MB)
Trainable params: 6423041 (24.50 MB)
Non-trainable params: 14714688 (56.13 MB)
_________________________________________________________________


In [24]:
model.compile(loss='binary_crossentropy',optimizer='rmsprop',metrics=['accuracy'])

In [29]:
history=model.fit_generator(train_gen_data,
                            steps_per_epoch=20000/224,
                            epochs=25,
                            validation_data=test_gen_data,
                            validation_steps=5000/224)

<ipython-input-29-ddbfb2aad1e3>:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history=model.fit_generator(train_gen_data,


Epoch 1/25
89/89 [==============================] - 41s 463ms/step - loss: 0.1879 - accuracy: 0.9319 - val_loss: 0.1973 - val_accuracy: 0.9154
Epoch 2/25
89/89 [==============================] - 61s 684ms/step - loss: 0.1520 - accuracy: 0.9438 - val_loss: 0.1523 - val_accuracy: 0.9386
Epoch 3/25
89/89 [==============================] - 61s 684ms/step - loss: 0.1628 - accuracy: 0.9333 - val_loss: 0.1801 - val_accuracy: 0.9256
Epoch 4/25
89/89 [==============================] - 41s 466ms/step - loss: 0.1767 - accuracy: 0.9285 - val_loss: 0.1753 - val_accuracy: 0.9300
Epoch 5/25
89/89 [==============================] - 41s 466ms/step - loss: 0.1660 - accuracy: 0.9250 - val_loss: 0.1742 - val_accuracy: 0.9270
Epoch 6/25
89/89 [==============================] - 40s 450ms/step - loss: 0.1636 - accuracy: 0.9444 - val_loss: 0.1868 - val_accuracy: 0.9228
Epoch 7/25
89/89 [==============================] - 41s 460ms/step - loss: 0.1678 - accuracy: 0.9299 - val_loss: 0.1460 - val_accuracy: 0.9406

KeyboardInterrupt: 

In [30]:
import matplotlib.pyplot as plt

In [31]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])

NameError: name 'history' is not defined